# 05 - Model Evaluation

Bu notebook'ta en iyi modeli detaylı olarak değerlendireceğiz.

## Hedefler:
- Kapsamlı model değerlendirmesi
- Feature importance analizi
- Farklı threshold değerlerini test etme
- Final model metrikleri

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score,
    f1_score
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## 1. Prepare Data and Train Model

In [ ]:
# Load and prepare data
df = pd.read_csv('../data/creditcard.csv')

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Train best model (using Random Forest as example)
final_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

print("Training final model...")
final_model.fit(X_train_smote, y_train_smote)
print("✅ Model trained")

## 2. Predictions and Basic Metrics

In [ ]:
# Predictions
y_pred = final_model.predict(X_test_scaled)
y_pred_proba = final_model.predict_proba(X_test_scaled)[:, 1]

# Metrics
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraud']))

roc_auc = roc_auc_score(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)

print(f"\nROC-AUC Score: {roc_auc:.4f}")
print(f"Average Precision Score: {avg_precision:.4f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted Normal', 'Predicted Fraud'],
    y=['Actual Normal', 'Actual Fraud'],
    text=cm,
    texttemplate='%{text}',
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix',
    height=500
)
fig.show()

# Calculate metrics from confusion matrix
tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")
print(f"\nFalse Positive Rate: {fp/(fp+tn):.4f}")
print(f"False Negative Rate: {fn/(fn+tp):.4f}")

## 3. ROC and PR Curves

In [ ]:
# ROC Curve
fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_proba)

# Precision-Recall Curve
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ROC Curve', 'Precision-Recall Curve')
)

# ROC Curve
fig.add_trace(
    go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC (AUC={roc_auc:.4f})', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(color='red', dash='dash')),
    row=1, col=1
)

# PR Curve
fig.add_trace(
    go.Scatter(x=recall, y=precision, mode='lines', name=f'PR (AP={avg_precision:.4f})', line=dict(color='green')),
    row=1, col=2
)

fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
fig.update_yaxes(title_text="True Positive Rate", row=1, col=1)
fig.update_xaxes(title_text="Recall", row=1, col=2)
fig.update_yaxes(title_text="Precision", row=1, col=2)

fig.update_layout(height=500, showlegend=True)
fig.show()

## 4. Feature Importance

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features:")
print(feature_importance.head(15))

# Visualize
fig = go.Figure()

fig.add_trace(go.Bar(
    x=feature_importance['importance'][:20],
    y=feature_importance['feature'][:20],
    orientation='h',
    marker_color='steelblue'
))

fig.update_layout(
    title='Top 20 Feature Importances',
    xaxis_title='Importance',
    yaxis_title='Features',
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

## 5. Threshold Analysis

In [ ]:
# Test different thresholds
thresholds = np.arange(0.1, 1.0, 0.1)
threshold_results = []

for threshold in thresholds:
    y_pred_thresh = (y_pred_proba >= threshold).astype(int)
    
    cm = confusion_matrix(y_test, y_pred_thresh)
    tn, fp, fn, tp = cm.ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    threshold_results.append({
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'fp': fp,
        'fn': fn
    })

threshold_df = pd.DataFrame(threshold_results)
print(threshold_df)

In [ ]:
# Visualize threshold impact
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=threshold_df['threshold'],
    y=threshold_df['precision'],
    mode='lines+markers',
    name='Precision',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=threshold_df['threshold'],
    y=threshold_df['recall'],
    mode='lines+markers',
    name='Recall',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=threshold_df['threshold'],
    y=threshold_df['f1'],
    mode='lines+markers',
    name='F1-Score',
    line=dict(color='green')
))

fig.update_layout(
    title='Metrics vs Threshold',
    xaxis_title='Threshold',
    yaxis_title='Score',
    height=500
)
fig.show()

# Find optimal threshold (max F1)
optimal_idx = threshold_df['f1'].idxmax()
optimal_threshold = threshold_df.loc[optimal_idx, 'threshold']
print(f"\n🎯 Optimal Threshold (max F1): {optimal_threshold:.2f}")
print(f"F1-Score at optimal threshold: {threshold_df.loc[optimal_idx, 'f1']:.4f}")

## 6. Final Model Summary

In [ ]:
# Create summary
summary = f"""
═══════════════════════════════════════════════════════
           FINAL MODEL EVALUATION SUMMARY
═══════════════════════════════════════════════════════

Model: Random Forest Classifier

Performance Metrics:
  • ROC-AUC Score:          {roc_auc:.4f}
  • Average Precision:      {avg_precision:.4f}
  • Optimal Threshold:      {optimal_threshold:.2f}

Confusion Matrix (at 0.5 threshold):
  • True Negatives:         {tn}
  • False Positives:        {fp}
  • False Negatives:        {fn}
  • True Positives:         {tp}

Top 3 Important Features:
  1. {feature_importance.iloc[0]['feature']}: {feature_importance.iloc[0]['importance']:.4f}
  2. {feature_importance.iloc[1]['feature']}: {feature_importance.iloc[1]['importance']:.4f}
  3. {feature_importance.iloc[2]['feature']}: {feature_importance.iloc[2]['importance']:.4f}

═══════════════════════════════════════════════════════
"""

print(summary)

## 7. Summary

### Model Evaluation Complete:
- Comprehensive performance metrics calculated
- Feature importance analyzed
- Optimal threshold identified
- Model ready for deployment

### Next Steps:
1. Create final pipeline
2. Save model for deployment
3. Build deployment application

In [ ]:
print("✅ Model evaluation completed!")
print("\nNext: Run 06_final_pipeline.ipynb to create the final pipeline")